# Numeric Integration

What does the reaction progress of enzyme kinetics look like over a long stretch of time, far past the initial rate from which we have not yet dared stray? The math is similar to the case of the steady-state kinetic scheme above. However, in the Michaleis-Menten (MM) system, one of the reactants, the enzyme, is present at limiting quanties ($[E]_t \ll [S]_t$) and this case gives rise to the famous MM equation.

$$-\frac{d}{dt}[S] = k_{cat}[E]_t\frac{[S]_t}{K_M+[S]_t}$$

This equation can be integrated analytically using advanced mathematical methods that you will likely never care to learn. I certainly have no clue how to integrate anything but the simplest equations. So, yes, you will never use calculus again in your life but only because others have done the calculus for you already. Understanding the basics of calculus is important if you are to properly stand on the shoulders of mathematicians. The analytical integration was demonstrated in the previous notebook.

In this notebook will not use analytical methods but instead use the numerical algorithms encapsulated the Python tools. We will use `scipy.integration.odeint()`.

## Using *odient*

The `scipy.integration.odeint()` function will perform a numeric integration of ordinary differential equations. It is simpler to set up than the analytical integration and, in well behaved cases, is very acurate. Let us perform a numerical integration of the MM system. We can then compare the results with the analytical method.

If we know the value of [S] at a given moment in time then we can easily calculate the rate of change using the above equation. There is only one place where we know the substrate concentration for sure and that is at time zero. So we start there, calculate the rate of change, calculate the change after an infinitesimal amount of time and then start again with the new concentration and repeat endlessly. Obviously there are mathematical approaches to this and they all use various methods to minimize the error caused when the time step used is anything greater than $\frac{1}{\infty}$.

So we know that the numerical approach should be close to the true answer but can never be exact. The analytical method is exact but, as we saw, is very difficult with some kinds of equations. The code below defines a function for the rate of change in copncentration of substrate according to the MM equation. Then we use `scipy.integration.odeint()` to calculate the concentration of substrate over time using numerical integration methods.

In [1]:
#######################
### import packages ###
#######################

import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

def rates(c, t, E, kcat, Km):
    return -kcat * E * c / (c + Km)


############################################
### Create a function that describes the ###
### rates of all reactions in the system ###
############################################

E_conc = 1e-6
KM_value = 0.005          ### set KM value
kcat_value = 1E2          ### set Vmax value
S0_value = .05            ### set initial substrate concentration value

Vmax_value = E_conc * kcat_value

time_end = 1200           ### The end time
n_points = 100            ### number of points - increase if needed



#########################################
### Create series of time points      ###
#########################################

dt = .1                                    ### time step, delta t
tmax = time_end                            ### max time, s
t_line = np.linspace(0, tmax, num=int(tmax/dt)) ### time vector (list of time points)

xmax = time_end           ### change these as you wish
ymax = S0_value

##########################
### integrate
##########################

initial = S0_value
cc = odeint(rates, initial, t_line, args=(E_conc, kcat_value, KM_value))

##########################
### plot x and y data
##########################

x = t_line; xmax = tmax
y = cc

plt.rcdefaults()     ### resets the plot defaults so we always start in the same place
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(4,3))  

ax.plot( x, y,
         linestyle = '-',            ### use a line between points
         linewidth='0.5',            ### make the line thin
         color = 'black',            ### the line is black
         zorder = 0                  ### everything is in layer 0 (the bottom layer in this case)
        )
ax.set( xlabel= r"$t\;/\;s$", 
        ylabel=r"$[S] \; / \; mole\,L^{-1}$",
        title = "MM Reaction",
        xlim=[0, xmax],                  
        ylim=[-0.0001, ymax]      
       )

ax.spines[['top','right','bottom','left']].set_visible(False)   ### Make all spines invisible
ax.spines[['left','bottom']].set_position(('axes',-0.05))       ### space spines a little away from 'axes' (plot box)

#########################################
### Output the plot
#########################################

plt.savefig("plot.pdf", bbox_inches='tight')     ### export the plot as this
plt.show() 

## Comparing with Anaytical Integration

Below I have stolen the code from the previous notebook to calculate a set of x and y values using the analytical integration. When we are done below we will have a set of numerically integrated substrate concentrations with time (`cc` vs `t` from the code block above) and a set of anaytically calculated concentrations (`c` vs `t`) as calculated in the code block below.  The `t` series will be the same in both data sets.

Then I will plot the difference between the two sets of data. How closely does numerical integration match the truth of analytical integration in this case?



In [2]:
import sympy as sym

t = sym.symbols('t')           ### create x as a 'symbol', not a variable
Vmax = sym.symbols('V_{max}')  ### create k as a 'symbol'
St = sym.symbols('S_t')        ### create At as a 'symbol'
S0 = sym.symbols('S_0')        ### create A0 as a 'symbol'
KM = sym.symbols('K_M')

xt = sym.Function('x_t')       ### create x as a 'function', not a variable

lhs = sym.Derivative(-St, t)   ### Using Derivative function to get differential of A(t) w.r.t. t
rhs = Vmax*(St/(KM+St))

diffeq = sym.Eq(lhs, rhs)                   ### create a sympy equation
diffeq = diffeq.subs({St: (S0 - xt(t))})    ### Substitute the term with S0-x
res = sym.dsolve(diffeq, ics={xt(0): 0})    ### Solve the differential equation. Initial condition is x(t) = 0 when t = 0

eq = res.subs(xt(t), S0-St)            ### substitute x for So - St
eq = sym.simplify(eq)                  ### Simplify the result
eq = sym.Eq(eq.lhs - S0, eq.rhs - S0)  ### Subtract S0 from both sides of the equation
eq = sym.Eq(-eq.lhs, -eq.rhs)          ### take the negative of both sides of the equation 

print("The integrated rate law for the MM equation")
display(eq)                         

##############################################################################
##############################################################################
##############################################################################

from matplotlib import pyplot as plt
import numpy as np

##########################
### create function 
##########################

f = sym.lambdify([t, S0, KM, Vmax], eq.rhs)   ### create function in terms of t, S0, KM and Vmax

##########################
### Set up parameters
##########################

E_conc = 1e-6
KM_value = 0.005          ### set KM value
kcat_value = 1E2          ### set Vmax value
S0_value = .05            ### set initial substrate concentration value

Vmax_value = E_conc * kcat_value

time_end = 1200           ### The end time
n_points = 100            ### number of points - increase if needed

xmax = time_end       ### change these as you wish
ymax = S0_value

##########################
### calculate x and y data
##########################


### the time points
                                                ### t_line comes from the first code block above
c = f(t_line, S0_value, KM_value, Vmax_value)   ### Calculate y according to the function
c = np.real(c)                                  ### the Lambert function produces imaginary numbers. Use on the real component

cc = cc.flatten() 
c = c.flatten() 
diff = 100*(cc - c)/c   ### % error  



The integrated rate law for the MM equation


Eq(S_t, K_M*LambertW(S_0*exp((S_0 - V_{max}*t)/K_M)/K_M))

## Plot the Difference

Below we will take the `cc` series from the numerical method and the `c` series from the analytical method caluclated at the same time points and plot the difference. As we can see there is no difference. Inspecting the result of the difference calculation reveals a very tiny difference.

We only see errors of any detectable importance when the value of [S] is near zero. very small errors can be large compared to very small numbers. Even in that region, error was less that 3%.

The numerical integration produced the same result as the analytical in this case. And the numerical method was far simpler to set up.

Which would you rather use?

In [3]:
plt.rcdefaults()                    ### resets the plot defaults so we always start in the same place
fig, ax1 = plt.subplots(nrows=1, ncols=2, figsize=(8,3))  

x = t_line
y = diff  ### Percent error

#########################################
### Plot the line using the x_fit, y_fit points
#########################################

ax1[0].plot(x, y, 
        linestyle = '-',            ### use a line between points
        linewidth='0.5',            ### make the line thin
        color = 'black',            ### the line is black
        zorder = 0                  ### everything is in layer 0 (the bottom layer in this case)
        )

#########################################
### Add more style
#########################################
        

ax1[0].set(ylabel=r"$\% \; \Delta [S]\; $", 
        xlabel=r"$time\; /\; min$",
        title = "% Difference Plot",
        xlim=[None, None],      
        ylim=[-5, 5])

ax1[0].spines[['top','right','bottom','left']].set_visible(False)   ### Make all spines invisible
ax1[0].spines[['left','bottom']].set_position(('axes',-0.02))   ### space spines a little away from 'axes' (plot box)


ax2 = ax1[0].twinx()

ax2.plot(t_line, c,
        linestyle = '-',            ### use a line between points
        linewidth='5',            ### make the line thin
        color = 'white',            ### the line is black
        zorder = 0 )                 ### everything is in layer 0 (the bottom layer in this case)
ax2.plot(t_line, c,
        linestyle = '-',            ### use a line between points
        linewidth='0.5',            ### make the line thin
        color = 'black',            ### the line is black
        zorder = 0 )                 ### everything is in layer 0 (the bottom layer in this case)
       
ax2.text(900,0.005,"$[S]$")
ax1[0].text(750, 1.0, "$\%\, \Delta \,[S]$")


ax2.set(ylabel=r"$[S]\; / \;M$", 
        xlim=[None, None],      
        ylim=[-0.001, None])

ax2.spines[['top','right','bottom','left']].set_visible(False)   ### Make all spines invisible
ax2.spines[['right']].set_position(('axes',1.02))   ### space spines a little away from 'axes' (plot box)


######################################################################################

x = t_line
y = (cc-c) * 1E7
#########################################
### Plot the line using the x_fit, y_fit points
#########################################

ax1[1].plot(x, y, 
        linestyle = '-',            ### use a line between points
        linewidth='0.5',            ### make the line thin
        color = 'black',            ### the line is black
        zorder = 1                  ### everything is in layer 0 (the bottom layer in this case)
        )

ax1[1].text(900,0.2,"$\Delta \,[S]$")

#########################################
### Add more style
#########################################
        

ax1[1].set(ylabel=r"$\Delta \,[S]\; /\;10^{-7} \; M$", 
        xlabel=r"$time\; /\; min$",
        title = "Difference Plot",
        xlim=[None, None],      
        ylim=[-1, 1])

ax1[1].spines[['top','right','bottom','left']].set_visible(False)   ### Make all spines invisible
ax1[1].spines[['left','bottom']].set_position(('axes',-0.02))   ### space spines a little away from 'axes' (plot box)


ax3 = ax1[1].twinx()

ax3.plot(t_line, c,
        linestyle = '-',            ### use a line between points
        linewidth='5',            ### make the line thin
        color = 'white',            ### the line is black
        zorder = 3 )                 ### everything is in layer 0 (the bottom layer in this case)

ax3.plot(t_line, c,
        linestyle = '-',            ### use a line between points
        linewidth='0.5',            ### make the line thin
        color = 'black',            ### the line is black
        zorder = 3 )                 ### everything is in layer 0 (the bottom layer in this case)

ax3.text(900,0.005,"$[S]$")

ax3.set(ylabel=r"$[S]\; / \;M$", 
        xlim=[None, None],      
        ylim=[-0.001, None])

ax3.spines[['top','right','bottom','left']].set_visible(False)   ### Make all spines invisible

ax3.spines[['right']].set_position(('axes',1.02))   ### space spines a little away from 'axes' (plot box)



fig.tight_layout()  # otherwise the right y-label is slightly clipped
#########################################
### Output the plot
#########################################

plt.savefig("plot.pdf")     ### export the plot as this
plt.show()                                       ### display the plot in this notebook

